# Show restaurants and select the best one

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# hide warnings. `gbq.read_gbq()` gives some
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../scrape_save_search')

import load_data

In [ ]:
comments = load_data.load_comments()

In [ ]:
restaurants, existing_tag_ids, elastic_tag_ids, image_tag_ids = load_data.load_restaurants()

In [ ]:
restaurants.shape

# Top 5 for existing tags

In [ ]:
(
    restaurants
    .loc[lambda r: r['existing']]
    [['Name', 'Food rating', 'Price quality','Noise level', 'Waiting time']]
    .head()
)

# Elasticsearch tags

In [ ]:
(
    restaurants
    .loc[lambda r: r['elastic']]
    [['Name', 'Food rating', 'Price quality', 'Noise level', 'Waiting time']]
    .head()
)

# Image search tags

In [ ]:
(
    restaurants
    .loc[lambda r: r['image']]
    [['Name', 'Food rating', 'Price quality', 'Noise level', 'Waiting time']]
    .head()
)

# Venn diagram

In [ ]:
from matplotlib_venn import venn3
venn3([set(existing_tag_ids), set(elastic_tag_ids), set(image_tag_ids)], 
      set_labels = ('Existing tags', 'Elasticsearch tags', 'Image tags'))
plt.title("Different sources for hamburger tags", fontsize=16);

In [ ]:
(
    restaurants
    .loc[lambda r: r['image'] & r['elastic'] & r['existing']]
)

# Plot on map

In [ ]:
import collections
Coords = collections.namedtuple('Coordinates', 'lat lon')
coords_gdd = Coords(lat=52.352379, lon=4.912933)

In [ ]:
import geopy.distance
import folium
from folium.features import DivIcon

In [ ]:
def plot_map(df, coords_center, range_in_km=1, min_rating=8, zoom_start=15):
    m = folium.Map(location=list(coords_center), zoom_start=zoom_start)
    df = df.rename(columns={'Food rating': 'Food_rating'})
    for idx, row in df.iterrows():
        if row['Food_rating'] < min_rating:
            continue
        if geopy.distance.vincenty(coords_center, (row['info_lat'], row['info_lon'])).km > range_in_km:
            continue
        for tag_type, color in zip(['existing', 'elastic', 'image'], ['#DDB997', '#99C799', '#99B7DA']):
            if row[tag_type]:
                folium.map.Marker([row['info_lat'], row['info_lon']], 
                                  icon=DivIcon(html=f'<div title="{row.Name}", style="font-size: 15pt; display: inline-block; background-color: white; border-style: solid; padding:2px; border-color:{color}";>{row.Food_rating}</div>')
                                 ).add_to(m)

    folium.map.Marker(
        list(coords_center),
        icon=DivIcon(
            html='<div style="font-size: 15pt; display: inline-block; background-color:#5DBCD2; color:white;  border-style: solid; border-color:#5DBCD2"> GDD </div>',
            )
        ).add_to(m)
    return m

In [ ]:
# Hoover over rating with mouse for restaurant name
plot_map(restaurants, coords_gdd, range_in_km=5, min_rating=9)